In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import warnings; warnings.simplefilter('ignore')
import matplotlib.pyplot as plt
import mglearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import (confusion_matrix,precision_score,recall_score,f1_score,
    roc_curve,roc_auc_score,precision_recall_curve,accuracy_score,classification_report)

In [2]:
def read_xyz(fileName):
    
    xyz_coordinates = [] #put xyz in an array
    
    with open(fileName,"r") as file:
        for line_number,line in enumerate(file):
            x,y,z = line.split()
            
            xyz_coordinates.append([int(x),int(y),float(z)])
    return xyz_coordinates

In [3]:
groundwater_map = read_xyz("GRW_MBS_50m.xyz")

In [4]:
#display in dataframe 
#add cplumns of name
my_array = np.array(groundwater_map)

df = pd.DataFrame(my_array, columns = ['XKoordinat','YKoordinat','Depth'])



Now we have a table with x,y coordinates and depths. Next step is to copmare with csv file coordinates and get a column to state the ground water level.

In [5]:
data = pd.read_csv("Energi_Viborg_Dandas_data.csv")

#drop columns not needed after asking the company about the meaning of these features

columns_to_be_removed = [ 'mslink','LedningID','Dobbeltled','EjerKompon','SystemKode','KategoriAf','DatoUdf']
data=data.drop(columns_to_be_removed,axis='columns')

# in the column DatoSaneri is the date of repairing and if there is no date it means it is not repaired

data['DatoSaneri'].fillna(0, inplace=True)

In [6]:
# take only the pipes that are broken(by TV insection) now and the repaired ones

data_with_TVObsAndSaneri = data[data['TVObsKode'].isin([1]) | data['DatoSaneri'] > 0]

get matched depth withground water of broken pipes

In [7]:
def get_matched_depth(select_x,select_y):
#     select_x = data['XKoordinat']
#     select_y = data['YKoordinat']
    #select_y = data.loc[data['XKoordinat'] == select_x]['YKoordinat'].values[0]
    length = data_with_TVObsAndSaneri.loc[data_with_TVObsAndSaneri['XKoordinat'] == select_x]['Laengde'].values[0]
    angle = data_with_TVObsAndSaneri.loc[data_with_TVObsAndSaneri['XKoordinat'] == select_x]['Fald'].values[0]
#     length=data['Laengde']
#     angle = data['Fald']
    #calculate another point by length:
    end_x = select_x+ (length * np.cos(angle))
    end_y = select_y+ (length * np.sin(angle))
    if(end_x > select_x):
        max_x=end_x
        min_x=select_x
    else:
        min_x=end_x
        max_x=select_x   
    if(end_y > select_y):
        max_y=end_y
        min_y=select_y
    else:
        min_y=end_y
        max_y=select_y   


    matched_depth_col = df.loc[(df['XKoordinat'] <= max_x)&(df['XKoordinat'] >= min_x)
    &(df['YKoordinat'] <= max_y)&(df['YKoordinat'] >= min_y)]['Depth']
    #test if there is a value
    if(matched_depth_col.size > 0):
        matched_depth = matched_depth_col.values[0]
    else:
        matched_depth = np.NaN
    
    return matched_depth

In [8]:
def add_depth(datacopy):
    select_x = datacopy['XKoordinat']
    select_y = datacopy['YKoordinat']
    return get_matched_depth(select_x,select_y)
data_with_TVObsAndSaneri['Depth'] = data_with_TVObsAndSaneri.apply(add_depth,axis =1)
data_with_TVObsAndSaneri    
        

,ID,XKoordinat,YKoordinat,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,TransportK,Funktionsk,TVObsKode,DatoOprett,DatoOpdate,DatoSaneri,Depth
36,87810,529911.05,6252443.83,34.720000,33.480000,64.88,19.112207,300.0,1.0,1939.0,1,0,0.0,2010,2014,1997.0,NaN
42,87832,530405.37,6252578.04,39.460000,39.160000,91.75,3.269755,400.0,1.0,1939.0,1,0,1.0,2010,2014,0.0,2.314121
43,87834,530493.05,6252579.67,39.710000,39.480000,87.69,2.622876,300.0,1.0,1939.0,1,0,1.0,2010,2014,0.0,NaN
64,87901,530791.62,6252572.03,40.550000,40.080000,52.11,9.019382,250.0,1.0,1945.0,1,0,1.0,2010,2014,0.0,4.478954
65,87903,530857.05,6252552.13,40.380000,40.550000,68.39,-2.485744,250.0,1.0,1945.0,1,0,1.0,2010,2014,0.0,5.462685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23957,222195,500582.01,6260561.72,11.940000,10.550000,60.32,23.043767,350.0,1.0,1968.0,1,0,1.0,2018,2018,0.0,9.960063
24062,222448,544401.67,6256588.02,25.720000,22.820000,44.37,65.359477,160.0,4.0,2017.0,1,0,1.0,2018,2018,0.0,NaN
24073,222942,530696.61,6245426.58,28.490000,27.730000,84.29,9.016491,200.0,1.0,1995.0,1,0,1.0,2018,2018,0.0,NaN
24090,222967,530819.00,6245345.46,27.162706,27.902884,18.82,-39.329328,315.0,4.0,1965.0,1,0,1.0,2018,2018,0.0,NaN


In [9]:
data_with_TVObsAndSaneri_Groundwater = data_with_TVObsAndSaneri.dropna()


handle unbroken pipe by adding groundwater depth and get around 619 rows randomly

In [10]:
data_not_broken = data[~data['TVObsKode'].isin([0]) | data['DatoSaneri'] == 0]
data_not_broken = data_not_broken.sample(n=4000) 

In [11]:
def get_matched_depth_unbroken(select_x,select_y):
#     select_x = data['XKoordinat']
#     select_y = data['YKoordinat']
    #select_y = data.loc[data['XKoordinat'] == select_x]['YKoordinat'].values[0]
    length = data_not_broken.loc[data_not_broken['XKoordinat'] == select_x]['Laengde'].values[0]
    angle = data_not_broken.loc[data_not_broken['XKoordinat'] == select_x]['Fald'].values[0]
#     length=data['Laengde']
#     angle = data['Fald']
    #calculate another point by length:
    end_x = select_x+ (length * np.cos(angle))
    end_y = select_y+ (length * np.sin(angle))
    if(end_x > select_x):
        max_x=end_x
        min_x=select_x
    else:
        min_x=end_x
        max_x=select_x   
    if(end_y > select_y):
        max_y=end_y
        min_y=select_y
    else:
        min_y=end_y
        max_y=select_y   


    matched_depth_col = df.loc[(df['XKoordinat'] <= max_x)&(df['XKoordinat'] >= min_x)
    &(df['YKoordinat'] <= max_y)&(df['YKoordinat'] >= min_y)]['Depth']
    #test if there is a value
    if(matched_depth_col.size > 0):
        matched_depth = matched_depth_col.values[0]
    else:
        matched_depth = np.NaN
    
    return matched_depth

In [18]:
def add_depth_unbroken(datacopy):
    select_x = datacopy['XKoordinat']
    select_y = datacopy['YKoordinat']
    return get_matched_depth_unbroken(select_x,select_y)
data_not_broken['Depth'] = data_not_broken.apply(add_depth_unbroken,axis =1)
        

In [19]:
data_not_broken_Groundwater = data_not_broken.dropna()
data_not_broken_Groundwater =data_not_broken_Groundwater.sample(n=619)

In [20]:
frames = [data_not_broken_Groundwater,data_with_TVObsAndSaneri_Groundwater]
data = pd.concat(frames)

In [21]:
datacopy = data


# add  age column

#get current year
from datetime import date
now = date.today().year


def age_df(datacopy):

    if (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] > 0) :
        return (now - datacopy['DatoSaneri'])
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri']== 0):
        return (now - datacopy['anlag_aar'])
    elif (datacopy['TVObsKode'] == 0) and (datacopy['DatoSaneri'] > 0):
        return (now - datacopy['DatoSaneri'])
    elif (datacopy['TVObsKode']== 0) and (datacopy['DatoSaneri']== 0):
        return (now - datacopy['anlag_aar'])

datacopy['Age'] = datacopy.apply(age_df, axis = 1)

In [22]:
# add a column 'PipeStatus'
# 1 as broken and 0 as not broken

def broken_df(datacopy):

    if (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] < (datacopy['DatoOpdate'])) and (datacopy['DatoSaneri'] != 0):
        return 1
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri'] >= (datacopy['DatoOpdate'])) and (datacopy['DatoSaneri'] != 0):
        return 0
    elif (datacopy['TVObsKode'] == 1) and (datacopy['DatoSaneri']== 0):
        return 1
    elif (datacopy['TVObsKode'] == 0) and (datacopy['DatoSaneri'] > 0):
        return 0
    elif (datacopy['TVObsKode']== 0) and (datacopy['DatoSaneri']== 0):
        return 0

datacopy['PipeStatus'] = datacopy.apply(broken_df, axis = 1)

In [23]:
# data_fs= np.where(np.isnan(data_features))
# data_fs
print("Number of rows before removing NaNs: {}".format(datacopy.shape[0]))
datacopy = datacopy.dropna()
print("Number of rows after removing NaNs: {}".format(datacopy.shape[0]))

Number of rows before removing NaNs: 1238
Number of rows after removing NaNs: 1238


In [24]:
#drop columns not needed after adding new features

columns_to_be_removed = ['DatoOprett', 'DatoOpdate']
datacopy=datacopy.drop(columns_to_be_removed,axis='columns')
datacopy[0:-1]

,ID,XKoordinat,YKoordinat,fra_kote,til_kote,Laengde,Fald,DiameterIn,MaterialeK,anlag_aar,TransportK,Funktionsk,TVObsKode,DatoSaneri,Depth,Age,PipeStatus
13255,158854,521849.18,6251517.28,24.82,24.74,29.26,2.734108,300.0,1.0,1943.0,1,0,0.0,0.0,12.945337,78.0,0
2618,101958,546359.92,6262424.76,49.04,47.87,65.42,17.884439,160.0,4.0,1999.0,1,0,0.0,0.0,8.740685,22.0,0
23927,221915,529605.26,6255184.28,23.74,22.38,84.20,16.152019,250.0,4.0,2018.0,1,0,0.0,0.0,8.261030,3.0,0
16996,188135,515988.97,6243579.79,64.40,64.19,51.24,4.098361,300.0,1.0,1969.0,1,0,0.0,0.0,20.591042,52.0,0
15858,180964,539426.26,6248453.71,26.01,24.48,36.51,41.906327,200.0,4.0,2001.0,1,0,0.0,0.0,14.070298,20.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23825,221075,505295.19,6253688.34,41.00,40.24,77.12,9.854772,200.0,1.0,1979.0,1,0,1.0,0.0,14.129904,42.0,1
23835,221099,505487.19,6253719.27,40.77,40.36,57.64,7.113116,200.0,1.0,1979.0,1,0,1.0,0.0,14.236778,42.0,1
23838,221112,505553.82,6253730.01,41.26,40.78,67.49,7.112165,200.0,1.0,1979.0,1,0,1.0,0.0,15.404106,42.0,1
23841,221115,505234.88,6253724.12,42.49,41.53,109.15,8.795236,315.0,4.0,1992.0,1,0,1.0,0.0,16.049837,29.0,1


In [27]:
# val = datacopy.nunique()
# val

In [40]:
# creating features set and target

columns_to_be_removed = ['PipeStatus']
data_features= datacopy.drop(columns_to_be_removed,axis='columns')
columns_to_be_removed = ['ID','XKoordinat','YKoordinat','Depth','fra_kote','til_kote', 'Laengde','Fald','DiameterIn','MaterialeK','anlag_aar','TransportK','Funktionsk','TVObsKode','DatoSaneri','Age']
data_target=datacopy.drop(columns_to_be_removed,axis='columns')

In [41]:
# data_fs= np.where(np.isnan(data_features))
# data_fs
data = datacopy
print("Number of rows before removing NaNs: {}".format(data.shape[0]))
data = data.dropna()
print("Number of rows after removing NaNs: {}".format(data.shape[0]))

Number of rows before removing NaNs: 1238
Number of rows after removing NaNs: 1238


# Tuning hyperparameters with train-test split and grid search cross validation

In [43]:
# Divide the data into training and test
X_train, X_test, y_train, y_test = train_test_split(
    data_features, data_target, stratify=data_target, random_state=42)

In [44]:
#List Hyperparameters that we want to tune by cross validation

C = [100, 10, 1.0, 0.1, 0.01]
penalty = ['l1', 'l2']

#Convert to dictionary
hyperparameters = dict(C = C , penalty= penalty)

#Create new logistic object
logreg = LogisticRegression( solver = 'liblinear')

#Use GridSearch
clf = GridSearchCV(logreg, hyperparameters, cv=5,return_train_score=True, scoring="recall")

#Fit the model
best_model = clf.fit(X_train, y_train)

#Print The value of best Hyperparameters
print('Best C:', best_model.best_estimator_.get_params()['C'])
print('Best penalty:', best_model.best_estimator_.get_params()['penalty'])
    
print("Accuracy on training set: {:.3f}".format(best_model.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(best_model.score(X_test, y_test)))

Best C: 100
Best penalty: l1
Accuracy on training set: 0.986
Accuracy on test set: 0.958


In [45]:
#Create forest Object.
a = best_model.best_estimator_.get_params()['C']
b = best_model.best_estimator_.get_params()['penalty']

logreg = LogisticRegression(C = a, penalty= b, solver = 'liblinear')

#Create x and y variables.
x = data_features
y = data_target

#Split data into training and testing.
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=42)

#Training the model.
logreg.fit(X_train, y_train)

#Predict test data set.
y_pred =logreg.predict(X_test)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       239
           1       0.99      0.99      0.99        71

    accuracy                           0.99       310
   macro avg       0.99      0.99      0.99       310
weighted avg       0.99      0.99      0.99       310



0.9908656962696683

# Tuning hyperparameters with validation set split

In [46]:
# Divide the data into training, test and validation

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, random_state=43)
print("Size of training set:{}".format(X_train.shape[0]))
print("Size of validation set:{}".format(X_val.shape[0]))
print("Size of test set:{}".format(X_test.shape[0]))

Size of training set:696
Size of validation set:232
Size of test set:310


In [47]:
best_score = 0

for C in [100, 10, 1.0, 0.1, 0.01, 0.001]:
    for penalty in ['l1', 'l2']:
        
        # Learn the model
        logreg = LogisticRegression(penalty=penalty, C=C, solver = 'liblinear')
        logreg.fit(X_train, y_train)
        
        # Evaluate the model
        score = logreg.score(X_val, y_val)
        
        # If improvement, store score and parameter
        if score>best_score:
            best_score = score
            best_C = C
            best_penalty= penalty
                        


# Build a model on the combine training and valiation data
logreg = LogisticRegression(penalty= best_penalty, C =best_C, solver = 'liblinear')
logreg.fit(X_trainval, y_trainval)

print("Best best_C found: {}".format(best_C))
print("Best best_penalty found: {}".format(best_penalty))
print("Best score on validation set: {}".format(best_score))
print("Score on training/validation set: {}".format(logreg.score(X_trainval, y_trainval)))
print("Score on test set: {}".format(logreg.score(X_test, y_test)))

Best best_C found: 100
Best best_penalty found: l1
Best score on validation set: 0.9956896551724138
Score on training/validation set: 0.9989224137931034
Score on test set: 0.9967741935483871


In [48]:
#Predict test data set.
y_pred = logreg.predict(X_test)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       239
           1       1.00      0.99      0.99        71

    accuracy                           1.00       310
   macro avg       1.00      0.99      1.00       310
weighted avg       1.00      1.00      1.00       310



0.9929577464788732

# Tuning parameters with cross validation split

In [49]:
# Divide the data into training and test , no validation data

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)
print("Size of training set:{}".format(X_train.shape[0]))
print("Size of test set:{}".format(X_test.shape[0]))

Size of training set:696
Size of test set:310


In [50]:
best_score = 0

for C in [100, 10, 1.0, 0.1, 0.01, 0.001]:
    for penalty in ['l1', 'l2']:
        # Learn the model 
        logreg = LogisticRegression(penalty=penalty, C=C, solver = 'liblinear')
        
        # Perform cross validation
        scores = cross_val_score(logreg, X_trainval, y_trainval, cv=5)
        
        # Compute the mean score
        score = scores.mean()
        
        
        # If improvement, store score and parameter
        if score>best_score:
            best_score = score
            best_C = C
            best_penalty= penalty

# Build a model on the combine training and valiation data
logreg = LogisticRegression(penalty= best_penalty, C=best_C, solver = 'liblinear')
logreg.fit(X_trainval, y_trainval)

print("Best best_C found: {}".format(best_C))
print("Best best_penalty found: {}".format(best_penalty))
print("Best average score: {}".format(best_score))
print("Score on training/validation set: {}".format(logreg.score(X_trainval, y_trainval)))
print("Score on test set: {}".format(logreg.score(X_test, y_test)))

Best best_C found: 0.1
Best best_penalty found: l1
Best average score: 0.9913804126707353
Score on training/validation set: 0.9924568965517241
Score on test set: 0.9838709677419355


In [51]:
#Predict test data set.
y_pred = logreg.predict(X_test)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       239
           1       1.00      0.93      0.96        71

    accuracy                           0.98       310
   macro avg       0.99      0.96      0.98       310
weighted avg       0.98      0.98      0.98       310



0.9647887323943662

# Tuning hyperparameters with train-test split and Normalisation

In [52]:
# Divide the data into training and test
X_train, X_test, y_train, y_test = train_test_split(
    data_features, data_target, stratify=data_target, random_state=42)

# preprocessing using 0-1 scaling
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [53]:
#List Hyperparameters that we want to tune by cross validation

C = [100, 10, 1.0, 0.1, 0.01]
penalty = ['l1', 'l2']

#Convert to dictionary
hyperparameters = dict(C = C , penalty= penalty)

#Create new logistic object
logreg = LogisticRegression( solver = 'liblinear')

#Use GridSearch
clf = GridSearchCV(logreg, hyperparameters, cv=5,return_train_score=True, scoring="recall")

#Fit the model
best_model = clf.fit(X_train_scaled, y_train)

#Print The value of best Hyperparameters
print('Best C:', best_model.best_estimator_.get_params()['C'])
print('Best penalty:', best_model.best_estimator_.get_params()['penalty'])
    
print("Accuracy on training set: {:.3f}".format(best_model.score(X_train_scaled, y_train)))
print("Accuracy on test set: {:.3f}".format(best_model.score(X_test_scaled, y_test)))

Best C: 0.1
Best penalty: l1
Accuracy on training set: 1.000
Accuracy on test set: 1.000


In [54]:
#Create forest Object.
a = best_model.best_estimator_.get_params()['C']
b = best_model.best_estimator_.get_params()['penalty']

logreg = LogisticRegression(C = a, penalty= b, solver = 'liblinear')

#Training the model.
logreg.fit(X_train_scaled, y_train)

#Predict test data set.
y_pred =logreg.predict(X_test_scaled)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       238
           1       0.96      1.00      0.98        72

    accuracy                           0.99       310
   macro avg       0.98      0.99      0.99       310
weighted avg       0.99      0.99      0.99       310



0.9936974789915967

# Tuning hyperparameters with train-test split and standardization

In [55]:
# Divide the data into training and test
X_train, X_test, y_train, y_test = train_test_split(
    data_features, data_target, stratify=data_target, random_state=42)

# preprocessing using zero mean and unit variance scaling
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [56]:
#List Hyperparameters that we want to tune by cross validation

C = [100, 10, 1.0, 0.1, 0.01]
penalty = ['l1', 'l2']

#Convert to dictionary
hyperparameters = dict(C = C , penalty= penalty)

#Create new logistic object
logreg = LogisticRegression( solver = 'liblinear')

#Use GridSearch
clf = GridSearchCV(logreg, hyperparameters, cv=5,return_train_score=True, scoring="recall")

#Fit the model
best_model = clf.fit(X_train_scaled, y_train)

#Print The value of best Hyperparameters
print('Best C:', best_model.best_estimator_.get_params()['C'])
print('Best penalty:', best_model.best_estimator_.get_params()['penalty'])
    
print("Accuracy on training set: {:.3f}".format(best_model.score(X_train_scaled, y_train)))
print("Accuracy on test set: {:.3f}".format(best_model.score(X_test_scaled, y_test)))

Best C: 0.1
Best penalty: l1
Accuracy on training set: 1.000
Accuracy on test set: 1.000


In [57]:
#Create forest Object.
a = best_model.best_estimator_.get_params()['C']
b = best_model.best_estimator_.get_params()['penalty']

logreg = LogisticRegression(C = a, penalty= b, solver = 'liblinear')

#Training the model.
logreg.fit(X_train_scaled, y_train)

#Predict test data set.
y_pred =logreg.predict(X_test_scaled)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       238
           1       0.96      1.00      0.98        72

    accuracy                           0.99       310
   macro avg       0.98      0.99      0.99       310
weighted avg       0.99      0.99      0.99       310



0.9936974789915967

# Tuning hyperparameters with validation set split and Normalisation

In [58]:
# Divide the data into training, test and validation

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, random_state=43)

# preprocessing using 0-1 scaling
scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)
X_trainval_scaled = scaler.transform( X_trainval)

print("Size of training set:{}".format(X_train_scaled.shape[0]))
print("Size of validation set:{}".format(X_val_scaled.shape[0]))
print("Size of test set:{}".format(X_test_scaled.shape[0]))

Size of training set:696
Size of validation set:232
Size of test set:310


In [59]:
best_score = 0

for C in [100, 10, 1.0, 0.1, 0.01, 0.001]:
    for penalty in ['l1', 'l2']:
        
        # Learn the model
        logreg = LogisticRegression(penalty=penalty, C=C, solver = 'liblinear')
        logreg.fit(X_train_scaled, y_train)
        
        # Evaluate the model
        score = logreg.score(X_val_scaled, y_val)
        
        # If improvement, store score and parameter
        if score>best_score:
            best_score = score
            best_C = C
            best_penalty= penalty
                        


# Build a model on the combine training and valiation data
logreg = LogisticRegression(penalty= best_penalty, C =best_C, solver = 'liblinear')
logreg.fit(X_trainval_scaled, y_trainval)

print("Best best_C found: {}".format(best_C))
print("Best best_penalty found: {}".format(best_penalty))
print("Best score on validation set: {}".format(best_score))
print("Score on training/validation set: {}".format(logreg.score(X_trainval_scaled, y_trainval)))
print("Score on test set: {}".format(logreg.score(X_test_scaled, y_test)))

Best best_C found: 10
Best best_penalty found: l1
Best score on validation set: 0.9956896551724138
Score on training/validation set: 0.9978448275862069
Score on test set: 0.9870967741935484


In [60]:
#Predict test data set.
y_pred = logreg.predict(X_test_scaled)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       239
           1       0.99      0.96      0.97        71

    accuracy                           0.99       310
   macro avg       0.99      0.98      0.98       310
weighted avg       0.99      0.99      0.99       310



0.9767811892274147

# Tuning hyperparameters with validation set split and standardization

In [61]:
# Divide the data into training, test and validation

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, random_state=43)

# preprocessing using zero mean and unit variance scaling
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)
X_trainval_scaled = scaler.transform( X_trainval)

print("Size of training set:{}".format(X_train_scaled.shape[0]))
print("Size of validation set:{}".format(X_val_scaled.shape[0]))
print("Size of test set:{}".format(X_test_scaled.shape[0]))

Size of training set:696
Size of validation set:232
Size of test set:310


In [62]:
best_score = 0

for C in [100, 10, 1.0, 0.1, 0.01, 0.001]:
    for penalty in ['l1', 'l2']:
        
        # Learn the model
        logreg = LogisticRegression(penalty=penalty, C=C, solver = 'liblinear')
        logreg.fit(X_train_scaled, y_train)
        
        # Evaluate the model
        score = logreg.score(X_val_scaled, y_val)
        
        # If improvement, store score and parameter
        if score>best_score:
            best_score = score
            best_C = C
            best_penalty= penalty
                        


# Build a model on the combine training and valiation data
logreg = LogisticRegression(penalty= best_penalty, C =best_C, solver = 'liblinear')
logreg.fit(X_trainval_scaled, y_trainval)

print("Best best_C found: {}".format(best_C))
print("Best best_penalty found: {}".format(best_penalty))
print("Best score on validation set: {}".format(best_score))
print("Score on training/validation set: {}".format(logreg.score(X_trainval_scaled, y_trainval)))
print("Score on test set: {}".format(logreg.score(X_test_scaled, y_test)))

Best best_C found: 100
Best best_penalty found: l1
Best score on validation set: 0.9956896551724138
Score on training/validation set: 1.0
Score on test set: 0.9903225806451613


In [63]:
#Predict test data set.
y_pred = logreg.predict(X_test_scaled)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       239
           1       1.00      0.96      0.98        71

    accuracy                           0.99       310
   macro avg       0.99      0.98      0.99       310
weighted avg       0.99      0.99      0.99       310



0.9788732394366197

# Tuning hyperparameters with cross validation split and Normalisation

In [64]:
# Divide the data into training and test , no validation data

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)

# preprocessing using 0-1 scaling
scaler = MinMaxScaler()
scaler.fit(X_train)

X_test_scaled = scaler.transform(X_test)
X_trainval_scaled = scaler.transform( X_trainval)


print("Size of training set:{}".format(X_trainval_scaled.shape[0]))
print("Size of test set:{}".format(X_test_scaled.shape[0]))

Size of training set:928
Size of test set:310


In [65]:
best_score = 0

for C in [100, 10, 1.0, 0.1, 0.01, 0.001]:
    for penalty in ['l1', 'l2']:
        # Learn the model 
        logreg = LogisticRegression(penalty=penalty, C=C, solver = 'liblinear')
        
        # Perform cross validation
        scores = cross_val_score(logreg, X_trainval_scaled, y_trainval, cv=5)
        
        # Compute the mean score
        score = scores.mean()
        
        
        # If improvement, store score and parameter
        if score>best_score:
            best_score = score
            best_C = C
            best_penalty= penalty

# Build a model on the combine training and valiation data
logreg = LogisticRegression(penalty= best_penalty, C=best_C, solver = 'liblinear')
logreg.fit(X_trainval_scaled, y_trainval)

print("Best best_C found: {}".format(best_C))
print("Best best_penalty found: {}".format(best_penalty))
print("Best average score: {}".format(best_score))
print("Score on training/validation set: {}".format(logreg.score(X_trainval_scaled, y_trainval)))
print("Score on test set: {}".format(logreg.score(X_test_scaled, y_test)))

Best best_C found: 100
Best best_penalty found: l1
Best average score: 0.992444056960186
Score on training/validation set: 1.0
Score on test set: 0.9967741935483871


In [66]:
#Predict test data set.
y_pred = logreg.predict(X_test_scaled)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       239
           1       1.00      0.99      0.99        71

    accuracy                           1.00       310
   macro avg       1.00      0.99      1.00       310
weighted avg       1.00      1.00      1.00       310



0.9929577464788732

# Tuning hyperparameters with cross validation split and standardization

In [67]:
# Divide the data into training and test , no validation data

X_trainval, X_test, y_trainval, y_test = train_test_split(data_features, data_target, random_state=42)

# preprocessing using zero mean and unit variance scaling
scaler = StandardScaler()
scaler.fit(X_train)

X_test_scaled = scaler.transform(X_test)
X_trainval_scaled = scaler.transform( X_trainval)


print("Size of training set:{}".format(X_trainval_scaled.shape[0]))
print("Size of test set:{}".format(X_test_scaled.shape[0]))

Size of training set:928
Size of test set:310


In [68]:
best_score = 0

for C in [100, 10, 1.0, 0.1, 0.01, 0.001]:
    for penalty in ['l1', 'l2']:
        # Learn the model 
        logreg = LogisticRegression(penalty=penalty, C=C, solver = 'liblinear')
        
        # Perform cross validation
        scores = cross_val_score(logreg, X_trainval_scaled, y_trainval, cv=5)
        
        # Compute the mean score
        score = scores.mean()
        
        
        # If improvement, store score and parameter
        if score>best_score:
            best_score = score
            best_C = C
            best_penalty= penalty

# Build a model on the combine training and valiation data
logreg = LogisticRegression(penalty= best_penalty, C=best_C, solver = 'liblinear')
logreg.fit(X_trainval_scaled, y_trainval)

print("Best best_C found: {}".format(best_C))
print("Best best_penalty found: {}".format(best_penalty))
print("Best average score: {}".format(best_score))
print("Score on training/validation set: {}".format(logreg.score(X_trainval_scaled, y_trainval)))
print("Score on test set: {}".format(logreg.score(X_test_scaled, y_test)))

Best best_C found: 10
Best best_penalty found: l1
Best average score: 0.9902877070619006
Score on training/validation set: 0.9989224137931034
Score on test set: 0.9967741935483871


In [69]:
#Predict test data set.
y_pred = logreg.predict(X_test_scaled)

#Checking performance our model with classification report.
print(classification_report(y_test, y_pred))

#Checking performance our model with ROC Score.
roc_auc_score(y_test, y_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       239
           1       1.00      0.99      0.99        71

    accuracy                           1.00       310
   macro avg       1.00      0.99      1.00       310
weighted avg       1.00      1.00      1.00       310



0.9929577464788732